# 1. 从零开始的Python实现

In [24]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

## 1.1 生成数据集

- 数据描述：拟生成一个包含1000个样本，2个特征的数据集，即
  $$\bold{X}\in\mathbb{R}^{1000\times2},~\bold{w}=[2,-3.4]^T,~b=4.2$$
- 生成规则：
  $$ \bold{y} = \bold{X}\bold{w} + b + \bold{\epsilon} $$
  *where* $\bold{X}\sim \mathcal{N}(0,1),~\epsilon \sim \mathcal{N}(0,0.01)$

In [34]:
# ----- 生成数据集 ------
def gen_data(w,b,sample_size):
    """ Rules: y = Xw + b + eps """
    X = torch.normal( 0 , 1 , (sample_size,len(w)) ) # 第三个参数指定参数形状 
    y = torch.matmul(X,w) + b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1)) 

true_w = torch.tensor([2,-3.4]) # 设定模拟的真实w
true_b = 4.2 # 设定模拟的真实b

features,labels = gen_data(true_w,true_b,1000) # 生成1000个样本，features是X，labels是y

# ----- 结果测试 ------
print(f'[Example] \nfeatures:',features[0],'\nlabel:',labels[0])

[Example] 
features: tensor([ 0.3647, -0.3397]) 
label: tensor([6.0735])


## 1.2 读取数据

In [73]:
# ----- 读取数据 ------
def data_iter(batch_size,features,labels):
    """生成一个小批量样本迭代器"""
    sample_size = len(features)  #获取样本总数
    indices = list(range(sample_size)) # 生成一个样本索引的列表
    random.shuffle(indices) # 将样本索引列表打乱，然后只取出打乱后的前batch_size个索引

    for i in range(0, sample_size, batch_size): # 这里的i相当于是一个batch的起始索引，每调用一次data_iter函数，就会向下确定一个这组batch的索引的起点标号
        batch_indices = torch.tensor(
            indices[i:min(i + batch_size, sample_size)] ) # 从起点i开始，抽取batch_size个样本，除非已经到达最后一个样本
        yield features[batch_indices], labels[batch_indices] # 生成一个batch的样本
    
# ----- 结果测试 ------
demo_batch_size = 2
iter = data_iter(demo_batch_size,features,labels)
print(*next(iter))

tensor([[ 0.5533,  1.1319],
        [ 0.4434, -0.6582]]) tensor([[1.4726],
        [7.3262]])


> 说明：上述python自带的默认迭代器的执行效率较低（例如其需要将数据全部加载到内存中等）。因而在实际使用中通常会用PyTorch等框架中提供的数据迭代器来更高效地读取数据。

## 1.3 参数初始化

在这里，我们初始化参数模型为：w为正态分布随机取样数据，b为0向量。

In [74]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True) # 见下方解释
b = torch.zeros(1, requires_grad=True)

**说明**：
- requires_grad=True 指示 PyTorch 在计算张量 w 的梯度时要跟踪它。也就是说，PyTorch 会记录所有与 w 相关的操作，以便在后续的反向传播过程中计算梯度。这对于训练神经网络非常重要，因为在训练过程中需要计算模型参数相对于损失函数的梯度，以便更新这些参数。

### 补充：自动微分

**说明：**
- 根据设计好的模型，系统会构建一个计算图 *(computational graph)*， 来跟踪计算是哪些数据通过哪些操作组合起来产生输出。
- 自动微分使系统能够随后反向传播梯度。 这里，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数 *（反向传播有关内容详见第四章）*。

这里通过对$y = 2\bold{x}^T\bold{x}$求导，来说明自动微分的过程。

**1. 初始化x**

In [79]:
# 初始化 x
import torch
x = torch.arange(4.0)
print(x)

tensor([0., 1., 2., 3.])


**2. 初始化梯度容器**

重要的是，我们不会在每次对一个参数求导时都分配新的内存。 因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。

In [82]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None
print(x.grad)

None


**3.计算y**

In [92]:
y = 2 * torch.dot(x, x)
print(y)

tensor(28., grad_fn=<MulBackward0>)


**4. 计算梯度**

数学上：$ y = 2\bold{x}^T\bold{x}  = 2x_1^2 + 2x_2^2 + ... + 2x_n^2 $，因而有：${\partial y}/{\partial \bold{x}} = [4x_1, 4x_2, ..., 4x_n]$。

特别地，$y$这里在点$\bold{x} = [0,1,2,3]^T$处取值，因此其梯度为$[0,4,8,12]^T$。这里通过PyTorch的自动微分功能来求解梯度。

In [93]:
y.backward() # 通过调用反向传播函数来自动计算y关于x每个分量的梯度
print(x.grad)

tensor([ 0.,  4.,  8., 12.])


再举一例，设$y = \sum x_i$，则有$\partial y/\partial \bold{x} = [1,1,...,1]^T$。
在PyTorch中，有：

In [94]:
x.grad.zero_() # 清除x的梯度(否则默认会累加)
y = x.sum()
y.backward()
print(x.grad)

tensor([1., 1., 1., 1.])


## 1.4 定义模型

这里涉及到Python的*广播机制*

In [75]:
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

## 1.5 定义Loss Function

In [96]:
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

## 1.6 定义优化算法

此处为Mini-batch SGD

In [105]:
def SGD(params, lr, batch_size):  #@save
    """小批量随机梯度下降
    params: 模型参数
    lr: 学习率
    batch_size: 批量大小
    """
    with torch.no_grad(): # 见下文详细介绍
        for param in params: # 随机梯度下降是对样本取样，但是每次iter都会更新全部参数
            # GD的更新公式是：param := param - lr * param.grad
            param -= lr * param.grad / batch_size # lr/batch_size是针对batch-size来规范学习率
            param.grad.zero_()  # 梯度清零

**说明：**

1. `torch.no_grad():`的作用是在该语句块中，不会对`requires_grad=True`的tensor进行求导.
2. `with ... `的作用是添加了一个上下文管理器，它可以创建一个上下文，执行一些代码，并在代码块结束时自动清理资源；在这里with torch.no_grad(): 创建了一个上下文，它告诉PyTorch在这个上下文中不要计算梯度。当代码块结束时，PyTorch会自动恢复梯度计算。

## 1.7 模型训练

大致的训练流程概括如下：

- 初始化模型参数
  
- 重复一下训练，直到收敛：
  - 计算梯度
  $$ g:= \nabla_{\boldsymbol{\theta}}\frac{1}{|\mathcal{B}|}\sum_{i\in\mathcal{B}}L(f(\boldsymbol{x}_i;\boldsymbol{\theta}),y_i) $$
  - 更新参数
  $$ \boldsymbol{\theta} := \boldsymbol{\theta} - \eta g $$

其中，每一个迭代周期就成为一个epoch。 我们通过调用data_iter()将整个数据集进行遍历（这里假设batch的划分是可以正好整除的）

In [110]:
# 超参数与模型设定
batch_size = 3
lr = 0.03
num_epochs = 10
net = linreg
loss = squared_loss

> 说明：（这里linreg,squared_loss都是上文提到的函数，由于上面#@save所以这里不用再单独定义）

In [111]:
for epoch in range(num_epochs): # 对于规定的epoch数，进行迭代

    # 迭代过程：抽样，根据样本进行GD更新
    for X, y in data_iter(batch_size, features, labels): # 通过data_iter函数，每次迭代都会生成一个batch的样本
        l = loss(net(X, w, b), y)  # 计算损失, loss=squared_loss, net=linreg
        l.sum().backward()  # 求和并反向传播，详见说明1
        SGD([w, b], lr, batch_size)  # 使用参数的梯度更新参数

    # 输出每次epoch的迭代结果（损失）
    with torch.no_grad(): # 防止梯度累加
        train_l = loss(net(features, w, b), labels) # 这里的w,b是正在迭代更新的参数，features与labels是全局的（而非batch）
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.000054
epoch 2, loss 0.000054
epoch 3, loss 0.000055
epoch 4, loss 0.000053
epoch 5, loss 0.000054
epoch 6, loss 0.000054
epoch 7, loss 0.000054
epoch 8, loss 0.000054
epoch 9, loss 0.000053
epoch 10, loss 0.000054


**说明：**
1. `l.sum().backward()`中，求和是因为，通过`loss`得到的损失函数是针对每个sample都有一个损失，故求和得到一个标量（这在统计中也是这样操作的）。这里也联系到，求导和求和的可交换性。

## 1.8 结果评估

In [ ]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

# 2. 线性回归的简洁实现

## 2.1 生成数据集

In [3]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 2.2 读取数据集

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays) # torch.utils.data 
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

**说明：**
- `TensorDataset`
  - 用于封装多个 PyTorch 张量的类。通常情况下，这些张量包含输入数据和对应的目标（标签）数据。
  - 将输入数据和目标数据进行一一对应的封装，使得在训练过程中可以方便地获取每个样本的输入和对应的目标。
  

- `DataLoader`
  - PyTorch 中的一个数据加载器类，用于批量地加载和迭代数据。
  - 在构建深度学习数据集时，通常会创建一个 TensorDataset 对象，将输入数据和目标数据传递给它，然后将这个对象传递给 DataLoader 以进行批量数据的加载和迭代。
  - `batch_size` 是将数据集划分成小批量（batch），并根据指定的 batch_size 和 shuffle 参数将数据加载到模型中进行训练或推理。
  - 如果 `is_train` 为 `True`，则洗牌数据，通常在训练时使用

检验是否读取成功

In [7]:
next(iter(data_iter))

[tensor([[-0.0612, -0.7375],
         [ 0.0455,  1.0493],
         [-1.6744,  2.1532],
         [-1.4166, -0.4632],
         [ 0.0025,  1.9530],
         [-0.1717,  0.4166],
         [-1.4073,  1.7884],
         [ 0.3628,  0.5727],
         [-0.5619,  0.5477],
         [-0.4401, -0.9981]]),
 tensor([[ 6.5812],
         [ 0.7108],
         [-6.4898],
         [ 2.9556],
         [-2.4348],
         [ 2.4432],
         [-4.7060],
         [ 2.9887],
         [ 1.2184],
         [ 6.7113]])]

## 2.3 定义模型

我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 

In [13]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # 2是输入特征数(x_i的个数)，1是输出特征数

print(net)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


## 2.4 初始化模型参数

-  我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。
-  我们还可以使用替换方法normal_和fill_来重写参数值。

In [15]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 2.5 定义损失函数

计算均方误差使用的是MSELoss类

In [19]:
loss = nn.MSELoss()

## 2.6 定义优化算法

In [20]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 2.7 模型训练

In [22]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y) # net(X)是模型的预测值，y是真实值
        trainer.zero_grad() # 梯度清零
        l.backward() # 反向传播计算梯度
        trainer.step() # 根据梯度更新参数，也就是SGD更新一步
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000105
epoch 2, loss 0.000107
epoch 3, loss 0.000105


## 2.8 模型评估

In [23]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0011,  0.0001])
b的估计误差： tensor([0.0008])
